In [ ]:
import pickle
import numpy as np

with open('/home/connor/main/development/waveform_compare/testing_scripts/MBHB_params.pkl', 'rb') as f:
    pmbhb = pickle.load(f)

p_index = 0
pMBHB = pmbhb[p_index]

print(pMBHB)


from ldc.common import tools


modes = [(2,2)]#, (2,1), (3,3), (3,2), (4,4), (4,3)]

def s_ldc2pycbc(mag, pol):
    return mag*np.cos(pol)

psi, incl = tools.aziPolAngleL2PsiIncl(pMBHB["EclipticLatitude"],
                                       pMBHB["EclipticLongitude"],
                                       pMBHB['InitialPolarAngleL'],
                                       pMBHB['InitialAzimuthalAngleL'])

psi = psi+np.pi


static_params = {'approximant': 'IMRPhenomD',
     'mass2': pMBHB['Mass2'],
     'inclination': incl,
     'tc': pMBHB['CoalescenceTime'],
     'polarization': psi,
     'spin1z': s_ldc2pycbc(pMBHB['Spin1'], pMBHB['PolarAngleOfSpin1']),
     'spin2z': s_ldc2pycbc(pMBHB['Spin2'], pMBHB['PolarAngleOfSpin2']),
     'coa_phase' : pMBHB['PhaseAtCoalescence'],
     'distance': pMBHB['Distance'],
     'ra': pMBHB['EclipticLatitude'],
     'dec': pMBHB['EclipticLongitude'],
     'mode_array':modes}


print(static_params)

m = pMBHB['Mass1'] #true mass1 in LDC Sangria
#print(m)

diff = m*0.1
low_m = m - diff
high_m = m + diff
print(low_m)
print(high_m)

m2 = pMBHB['Mass2']
diff = m2*0.1
low_m2 = m2 - diff
high_m2 = m2 + diff
print(low_m2)
print(high_m2)

tc = pMBHB['CoalescenceTime']

diff = tc*0.1
low_tc = tc - diff
high_tc = tc + diff
print(low_tc)
print(high_tc)

dist = pMBHB['Distance']
diff = dist*0.1
low_dist = dist - diff
high_dist = dist + diff
print(low_dist)
print(high_dist)


In [ ]:
data_config = """


[data]

instruments = LISA_A
trigger-time = 11485760
analysis-start-time = -655360
analysis-end-time = 655360

; strain settings
sample-rate = 0.2

; psd settings
psd-file=psd_inf_file.txt

; Frame file channel name for AET
frame-files = LISA_A:A_TDI.gwf
channel-name = LISA_A:LA:LA


"""

In [ ]:
model_config =      """


[model]
name = marginalized_phase
low-frequency-cutoff = 1e-6
high-frequency-cutoff = 1e-1

[variable_params]
; waveform parameters that will vary in MCMC
mass1 =
mass2 =
tc =
distance =

[static_params]
approximant=IMRPhenomD
;mass2=612485.5060299999
spin1z=0.26863190922667673
spin2z=-0.4215109787709388
;inclination=2.2517895222056112
polarization=1.2912226019238924
;coa_phase=1.2201968860015653
;distance=13470.983558972537
;ra=-0.30300442294174235
;dec=1.2925183861048521
;tc=11526944.921879262


[prior-mass1]
name = uniform
min-mass1 = 1190949.731387
max-mass1 = 1455605.227252

[prior-mass2]
name = uniform
min-mass2 = 551236.955427
max-mass2 = 673734.0566329999

[prior-tc]
name = uniform
min-tc = 11400000
max-tc = 11600000

[prior-distance]
name = uniform
;min-distance = 12123.885203075282
;max-distance = 14818.081914869792
min-distance = 1000
max-distance = 35000



"""

In [ ]:
sampler_config = """

[sampler]
name = dynesty
;ndim=4
nlive=150

"""

In [ ]:
!echo '{data_config}' > data_debug-config.ini
!cat data_debug-config.ini

In [ ]:
!echo '{model_config}' > model_debug-config.ini
!cat model_debug-config.ini

In [ ]:
!echo '{sampler_config}' > sampler_debug-config.ini
!cat sampler_debug-config.ini